# Feature Engineering - 0) Validation Set Split

As of Dec 02, 2018

In [12]:
## Google Storage Bucket 연결

OUTPUT_BUCKET_FOLDER = "gs://cap-18/output/" # 작업 결과물을 저장하는 path
DATA_BUCKET_FOLDER = "gs://cap-18/data/" # 작업을 위한 데이터를 저장해놓은 path

In [13]:
from pyspark.sql.types import * # 데이터 프레임 생성 + 형태(type)를 위한 모듈 로드
import pyspark.sql.functions as F # 내장 함수 활용을 위한 모듈 로드

## 데이터 로드

In [14]:
## 시간 정보를 변환하기 위한 사용자 정의 함수 생성

truncate_day_from_timestamp_udf = F.udf(lambda ts: int(ts / 1000 / 60 / 60 / 24), IntegerType()) # 정수 형태로 output을 반환

# cf.) ts / 1000 / 60 / 60 / 24: 밀리세컨드 단위를 1000, 60, 60, 24으로 나누면 일(day) 단위로 변환

In [17]:
## events_df 테이블 생성

# events_df 테이블의 구조(스키마) 사전 설정
events_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("uuid_event", StringType(), True),                    
                    StructField("document_id_event", IntegerType(), True),
                    StructField("timestamp_event", IntegerType(), True),
                    StructField("platform_event", IntegerType(), True),
                    StructField("geo_location_event", StringType(), True)]
                    )

# events_df 테이블의 구조에 맞춰 events.csv 파일 로드
events_df = spark.read.schema(events_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER + "events.csv") \
                .withColumn('day_event', truncate_day_from_timestamp_udf('timestamp_event')) \
                .alias('events')

SyntaxError: unexpected character after line continuation character (<ipython-input-17-789313a30f87>, line 14)

In [8]:
## promoted_content_df 테이블 생성

# promoted_content_df 테이블의 구조 사전 설정
promoted_content_schema = StructType(
                    [StructField("ad_id", IntegerType(), True),
                    StructField("document_id_promo", IntegerType(), True),                    
                    StructField("campaign_id", IntegerType(), True),
                    StructField("advertiser_id", IntegerType(), True)]
                    )

# promoted_content_df 테이블의 구조에 맞춰 promoted_content.csv 파일 로드
promoted_content_df = spark.read.schema(promoted_content_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"promoted_content.csv") \
                .alias('promoted_content')

NameError: name 'spark' is not defined

In [7]:
## clicks_train_df 테이블 생성

# clicks_train_df 테이블의 구조 사전 설정
clicks_train_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("ad_id", IntegerType(), True),                    
                    StructField("clicked", IntegerType(), True)]
                    )

# clicks_train_df 테이블의 구조에 맞춰 clicks_train.csv 파일 로드
clicks_train_df = spark.read.schema(clicks_train_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"clicks_train.csv") \
                .alias('clicks_train')

In [9]:
## clicks_train_joined_df 테이블 생성

clicks_train_joined_df = clicks_train_df \
                         .join(promoted_content_df, on='ad_id', how='left') \
                         .join(events_df, on='display_id', how='left')                         
clicks_train_joined_df.createOrReplaceTempView('clicks_train_joined')

In [10]:
## -- clicks_train_joined에서 일자별, 일정 비율로 데이터 샘플링하여 validation_set 생성

validation_display_ids_df = clicks_train_joined_df.select('display_id','day_event').distinct() \
                                .sampleBy("day_event", fractions={0: 0.2, 1: 0.2, 2: 0.2, 3: 0.2, 4: 0.2, \
                                                                5: 0.2, 6: 0.2, 7: 0.2, 8: 0.2, 9: 0.2, 10: 0.2, \
                                                               11: 1.0, 12: 1.0}, seed=0)   
validation_display_ids_df.createOrReplaceTempView("validation_display_ids")                                                                 

In [11]:
validation_set_df = spark.sql('''SELECT display_id, ad_id, uuid_event, day_event, timestamp_event,
                                        document_id_promo, platform_event, geo_location_event FROM clicks_train_joined t 
             WHERE EXISTS (SELECT display_id FROM validation_display_ids 
                           WHERE display_id = t.display_id)''')

In [ ]:
validation_set_df.show(2)

I/O를 최소한으로 유지하기 위해 컬럼 기반 저장 포맷인 파케이(Parquet)로 저장하자.

In [12]:
validation_set_gcs_output = "validation_set.parquet"
validation_set_df.write.parquet(OUTPUT_BUCKET_FOLDER+validation_set_gcs_output, mode='overwrite')

혹시 모를 일(?)을 위해 csv로도 저장해놓자.

In [ ]:
validation_set_df.toPandas().to_csv('validation_set_df.csv', index = False)
!gsutil cp 'validation_set_df' 'gs://cap-18/output/validation_set_df.csv'